##### `This file runs the BLSTM model. It needs to be run on paperspace gradient. Remember to put the dataset (in npy) in the current folder and change the dataset name if needed.`

Last updated on August 4th, 2022

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import metrics

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
#reference: https://www.tensorflow.org/guide/keras/train_and_evaluate

#for PCA
from sklearn.decomposition import PCA

#read in the data
dataset = np.load("regularOnly.npy", allow_pickle = True)
dataset = np.delete(dataset, 3889, axis=1)
dataset = np.delete(dataset, 3894, axis=1)

In [3]:
def train_test_split_by_speaker(data, test_size=0.2, pca=False, numPC=50):
    '''
    parameters:
    data: type to be decided, the dataset that we will train the model on
    test_size: the percentage of test data, by default 0.2
    pca: Bool. if to conduct PCA on the dataset, by default False
    numPC: number of principal components, by default 50
    
    return:
    train_x:
    train_y:
    test_x:
    test_y:
    '''

    #initial split of index for train and test 
    # testIndex = list(range(0,numOfVector))
    # trainIndex = list(range(numOfVector,data.shape[0]))
    
    #get the speaker name column
    name = data[:,1]
    nameList = []
    nameIndexDic = {}
    for i in range(name.shape[0]):
        try: 
            nameIndexDic[name[i]].append(i)
        except:
            nameList.append(name[i])
            nameIndexDic[name[i]] = [i]
    nameList = np.array(nameList)
    np.random.shuffle(nameList)

    numOfVector = round(len(nameList)*test_size)
    
    testIndex = []
    trainIndex = []
    #create index list for train and test 
    for j in range(len(nameList)):
        #first test dataset
        if j < numOfVector:
            testIndex = testIndex + nameIndexDic[nameList[j]]
        else: 
            trainIndex = trainIndex + nameIndexDic[nameList[j]]

    data = data[:,2:].astype(float)

    if pca:
        #norm the data
        for i in range(1, data.shape[1]):
            if np.std(data[:,i])==0:
                continue
            else:
                data[:,i] = (data[:,i] - np.mean(data[:,i]))/np.std(data[:,i])

        pcaModel = PCA(numPC)
        pcaData = pcaModel.fit_transform(data[:,1:])
        data = np.column_stack((data[:,0],pcaData))

    #split
    test = data[testIndex]
    train = data[trainIndex]
    
    test_x = test[:,1:].astype(float)
    test_y = test[:,0].astype(int)
    train_x = train[:,1:].astype(float)
    train_y = train[:,0].astype(int)

    return train_x, train_y, test_x, test_y, nameIndexDic

train_X, train_Y, test_X, test_Y, dic = train_test_split_by_speaker(dataset, pca=True,test_size=0.1)

(90,) (20,)


In [13]:
def run_model_n_times(n=10):
    '''
    parameter:
    n: int. the number of times of model training before calculating the statistics

    return:
    pd dataframe: the mean and an interval for n time of training
    '''
    #create the final statistics array
    final = []
    
    #initialize the model
    max_features = 100  # ALL THE FEATURES
    maxlen = 100  # Only consider the first 200 words of each movie review

    # Input for variable-length sequences of integers
    inputs = keras.Input(shape=(None,), dtype="int32")
    # Embed each integer in a 128-dimensional vector
    x = layers.Embedding(max_features, 128)(inputs)
    # Add 2 bidirectional LSTMs
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    # Add a classifier
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    # model.summary()
    
    
    for i in range(n):
        train_X, train_Y, test_X, test_Y = train_test_split(dataset, test_size=0.1)

        # test if there is empty result
        na = np.isnan(train_X)

        for i in range(na.shape[0]):
            for j in range(na.shape[1]):
                if bool(na[i,j])==True:
                    print(i,j)         

        train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=maxlen)
        test_X = keras.preprocessing.sequence.pad_sequences(test_X, maxlen=maxlen)
    
        #tfa.metrics.F1Score(num_classes=2)
        model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision(),keras.metrics.Recall(),keras.metrics.AUC()])
        result = model.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(test_X, test_Y))
        
        statistics = []
        index = 0
        for data in result.history.values():
            if index >= 6:
                statistics.append(data[-1]) 
            index += 1
        statistics.append(2*statistics[1]*statistics[2]/(statistics[1]+statistics[2]))
        final.append(statistics)
        
    final = pd.DataFrame(final, columns=["accuracy", "precision", "recall", "auc", "f1"])

    # final.form = [accuracy, precision, recall, auc, f1]
    return final.quantile([0.25, 0.5, 0.75])
    
        

In [15]:
print("\n\n\n\n\n", run_model_n_times(10))

Epoch 1/100
3/3 [==============================] - 9s 986ms/step - loss: 0.6922 - accuracy: 0.5682 - precision_9: 0.5952 - recall_9: 0.9259 - auc_9: 0.3685 - val_loss: 0.6675 - val_accuracy: 0.7727 - val_precision_9: 0.7727 - val_recall_9: 1.0000 - val_auc_9: 0.1471
Epoch 2/100
3/3 [==============================] - 0s 41ms/step - loss: 0.6817 - accuracy: 0.6136 - precision_9: 0.6136 - recall_9: 1.0000 - auc_9: 0.4284 - val_loss: 0.6375 - val_accuracy: 0.7727 - val_precision_9: 0.7727 - val_recall_9: 1.0000 - val_auc_9: 0.0882
Epoch 3/100
3/3 [==============================] - 0s 43ms/step - loss: 0.6713 - accuracy: 0.6136 - precision_9: 0.6136 - recall_9: 1.0000 - auc_9: 0.4763 - val_loss: 0.6093 - val_accuracy: 0.7727 - val_precision_9: 0.7727 - val_recall_9: 1.0000 - val_auc_9: 0.1000
Epoch 4/100
3/3 [==============================] - 0s 42ms/step - loss: 0.6607 - accuracy: 0.6136 - precision_9: 0.6136 - recall_9: 1.0000 - auc_9: 0.6634 - val_loss: 0.5966 - val_accuracy: 0.7727 - va